In [4]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import numpy as np
import math
import random
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import torch

ModuleNotFoundError: No module named 'torch'

In [2]:
def getSize(txt, font_size, font_name, max_width=224):
    font = ImageFont.truetype(font_name, font_size)
    testImg = Image.new('RGB', (1, 1))
    testDraw = ImageDraw.Draw(testImg)
    width, height = testDraw.textsize(txt, font)
    while width > max_width:
        font_size = font_size - 1
        font = ImageFont.truetype(font_name, font_size)
        testImg = Image.new('RGB', (1, 1))
        testDraw = ImageDraw.Draw(testImg)
        width, height = testDraw.textsize(txt, font)
    return font_size, width, height

In [39]:
def generate_image(path, name, text, dir_name, font_name, font_size, 
                   shift_val=30, size_red_val=60, im_suffix_start=1, no_of_images=10, 
                   test=False, base_only=False, word=False):
    colorText = "black"
    colorOutline = "red"
    colorBackground = "white"

    font_size, width, height = getSize(text, font_size, font_name)
    font = ImageFont.truetype(font_name, font_size)

    img = Image.new('RGB', (width+4, height+4), colorBackground)
    img = Image.new('RGB', (224, 224), colorBackground)
    d = ImageDraw.Draw(img)
    d.text((112 - (width / 2), 112 - (height / 2)), text, fill=colorText, font=font)
    
    if test:
        path = os.path.join(path, dir_name, 'test')
    else:
        path = os.path.join(path, dir_name, 'train')
        
    if not os.path.exists(path):
        os.makedirs(path)
    
    if not test or word:
        img.save(os.path.join(path, '{}_{}.jpg'.format(name, im_suffix_start)))
    
    if not base_only:
        for i in range(im_suffix_start + 1, im_suffix_start + no_of_images):
            shift = np.random.randint(-shift_val, shift_val)
            size_red = np.random.randint(-size_red_val, size_red_val)
            font = ImageFont.truetype(font_name, font_size - size_red)
            img = Image.new('RGB', (224, 224), colorBackground)
            d = ImageDraw.Draw(img)
            d.text((112 - (width / 2) + shift, 112 - (height / 2) + shift), text, fill=colorText, font=font)
            img.save(os.path.join(path, '{}_{}.jpg'.format(name, i)))

In [26]:
generate_image('/home/a38iqbal/cycle-gan-pytorch/font_images/', 
               'test.jpg', 
               'A', 
               'comic.ttf',
               40)

In [56]:
# with open('/home/a38iqbal/text_data/vocab.kos.txt') as f:
#     count = 1
#     maxi = 0
#     m = ''
#     for line in f:
#         word = line.strip()
#         generate_image('/Users/asifiqbal/Development/courses/ryerson/CP8309/text_data/', 
#                '{}.jpg'.format(str(count)), 
#                word, 
#                'Comic Sans MS.ttf',
#                40)
#         if len(word) > maxi:
#             maxi = len(word)
#             m = word
#         count += 1
#     print(maxi)
#     print(m)

40
openhttpwwwedwardsforprezcomdailykoshtml


In [20]:
# train image generation
count = 1
maxi = 0
m = ''
chars = [chr(x) for x in range(65, 91)]
for char in chars:
    generate_image('/media/external4T/a38iqbal/cycle_gan/font_images/train', 
           '{}'.format(char), 
           char, 
           'arial',
           'Arial.ttf',
           150)
print(maxi)
print(m)

0



In [21]:
# test image generation
count = 1
maxi = 0
m = ''
chars = [chr(x) for x in range(65, 91)]
for char in chars:
    generate_image('/media/external4T/a38iqbal/cycle_gan/font_images/test', 
           '{}'.format(char), 
           char, 
           'arial',
           'Arial.ttf',
           150,
           shift_val=30, size_red_val=60, im_suffix_start=1, no_of_images=10, test=True)
print(maxi)
print(m)

0



In [24]:
import matplotlib.font_manager
matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

['/usr/share/fonts/truetype/tlwg/Sawasdee-Bold.ttf',
 '/usr/share/fonts/truetype/tlwg/TlwgTypewriter-Oblique.ttf',
 '/usr/share/fonts/truetype/dejavu/DejaVuSans-BoldOblique.ttf',
 '/usr/share/fonts/truetype/tlwg/Norasi-Italic.ttf',
 '/usr/share/fonts/truetype/droid/DroidSansHebrew-Regular.ttf',
 '/usr/share/fonts/opentype/cabin/Cabin-Bold.otf',
 '/usr/share/fonts/truetype/msttcorefonts/arial.ttf',
 '/usr/share/fonts/truetype/freefont/FreeSerif.ttf',
 '/usr/share/fonts/truetype/comfortaa/Comfortaa-Light.ttf',
 '/usr/share/fonts/truetype/tlwg/Umpush-Bold.ttf',
 '/usr/share/fonts/truetype/kacst/KacstNaskh.ttf',
 '/usr/share/fonts/truetype/junicode/Junicode.ttf',
 '/usr/share/fonts/truetype/unfonts-core/UnGungseo.ttf',
 '/usr/share/fonts/truetype/dejavu/DejaVuSerifCondensed-Italic.ttf',
 '/usr/share/fonts/truetype/tlwg/Waree-Oblique.ttf',
 '/usr/share/fonts/truetype/lyx/msam10.ttf',
 '/usr/share/fonts/truetype/tlwg/Kinnari-Bold.ttf',
 '/usr/share/fonts/truetype/tlwg/TlwgTypo-Oblique.ttf',


In [44]:
dic = {}
with open('/media/external4T/a38iqbal/cycle_gan/vocab.kos.txt') as f:
    for line in f:
        word = line.strip()
        if len(word) <= 9:
            l = word[0]
            lis = dic.get(l, [])
            lis.append(word)
            dic[l] = lis

training_arial = []
training_times = []

test_arial = []
test_times = []

for key, val in dic.items():
    arial_part = random.sample(val, math.ceil(len(val) / 2))
    times_part = list(set(val) - set(arial_part))
    
    arial_train_sample = random.sample(arial_part, math.ceil(len(arial_part) / 6))
    arial_rest = list(set(arial_part) - set(arial_train_sample))
    arial_test_sample = random.sample(arial_rest, math.ceil(len(arial_rest) / 5))
    
    times_train_sample = random.sample(times_part, math.ceil(len(times_part) / 6))
    times_rest = list(set(times_part) - set(times_train_sample))
    times_test_sample = random.sample(times_rest, math.ceil(len(times_rest) / 5))
    
    training_arial += arial_train_sample
    test_arial += arial_test_sample
    
    training_times += times_train_sample
    test_times += times_test_sample

In [55]:
len(test_times)

492

In [49]:
# train word image generation
for word in training_arial:
    generate_image('/media/external4T/a38iqbal/cycle_gan/font_images/word', 
           '{}'.format(word), 
           word, 
           'arial',
           'Arial.ttf',
           150,
           base_only=True)

In [50]:
# test word image generation
for word in test_arial:
    generate_image('/media/external4T/a38iqbal/cycle_gan/font_images/word', 
           '{}'.format(word), 
           word, 
           'arial',
           'Arial.ttf',
           150,
           test=True,
           base_only=True,
           word=True)

In [53]:
# train word image generation
for word in training_times:
    generate_image('/media/external4T/a38iqbal/cycle_gan/font_images/word', 
           '{}'.format(word), 
           word, 
           'times',
           'Times_New_Roman.ttf',
           150,
           base_only=True)

In [56]:
# test word image generation
for word in test_times:
    generate_image('/media/external4T/a38iqbal/cycle_gan/font_images/word', 
           '{}'.format(word), 
           word, 
           'times',
           'Times_New_Roman.ttf',
           150,
           test=True,
           base_only=True,
           word=True)

In [3]:
with open('history_till_epoch_49.pkl', 'rb') as f:
    history = pickle.load(f)

ModuleNotFoundError: No module named 'torch'